# So far we have see the following:
    - How we can create a Database
    - How we can create a Table
    - How to load the data from a tempView to a Table 
    - We created Managed Table, and when we dropped the table:
        - Both the data and metadata got deleted

## Types of table:
    1. Managed 
    
        spark.sql('CREATE TABLE my_db_spark.orders \
               (order_id integer, \
                order_date string, \
                customer_id integer, \
                order_status string) \
                USING csv')
                
        spark.sql("INSERT INTO orders \
            SELECT * \
            FROM orders_view")
            
            
    2. External 
    
         spark.sql('CREATE TABLE my_db_spark.orders \
               (order_id integer, \
                order_date string, \
                customer_id integer, \
                order_status string) \
                USING csv \
                LOCATION '<S3:Path>')

## Create an `external` Table

    - We dont own the data 
    - We own ONLY the metadata 
    - We can not delete the data (as many others might be using the data) 
        - When we drop, it will DROP only the meta data 
        - Even if we use TRUNCATE, it would fail

In [57]:
spark.catalog.currentDatabase()

'default'

In [58]:
spark.sql('SHOW databases').show()

+--------------------+
|           namespace|
+--------------------+
|db_youtube_analytics|
| db_youtube_cleansed|
|      db_youtube_raw|
|             default|
|        dev_feedback|
+--------------------+



In [59]:
spark.sql('CREATE DATABASE IF NOT EXISTS my_db_spark')

23/05/14 02:28:15 INFO FileUtils: Creating directory if it doesn't exist: hdfs://ip-172-31-2-35.us-east-2.compute.internal:8020/user/spark/warehouse/my_db_spark.db


DataFrame[]

In [60]:
spark.sql('SHOW databases').show()

+--------------------+
|           namespace|
+--------------------+
|db_youtube_analytics|
| db_youtube_cleansed|
|      db_youtube_raw|
|             default|
|        dev_feedback|
|         my_db_spark|
+--------------------+



In [61]:
spark.sql('USE my_db_spark')

DataFrame[]

In [62]:
# spark.catalog.currentDatabase()
spark.sql("SELECT current_database()").show()

+------------------+
|current_database()|
+------------------+
|       my_db_spark|
+------------------+



In [67]:
spark.sql('SHOW TABLES').show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+



In [68]:
spark.sql("CREATE TABLE my_db_spark.orders \
           (order_id integer, \
            order_date string, \
            customer_id integer, \
            order_status string) \
            USING parquet \
            OPTIONS ('header'='true', 'inferSchema'='true') \
            LOCATION 's3://fcc-spark-example/dataset/2023/my_orders'")

DataFrame[]

In [69]:
spark.sql('SELECT * FROM my_db_spark.orders').show()

+--------+--------------------+-----------+---------------+
|order_id|          order_date|customer_id|   order_status|
+--------+--------------------+-----------+---------------+
|       1|2013-07-25 00:00:...|      11599|         CLOSED|
|       2|2013-07-25 00:00:...|        256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:...|      12111|       COMPLETE|
|       4|2013-07-25 00:00:...|       8827|         CLOSED|
|       5|2013-07-25 00:00:...|      11318|       COMPLETE|
|       6|2013-07-25 00:00:...|       7130|       COMPLETE|
|       7|2013-07-25 00:00:...|       4530|       COMPLETE|
|       8|2013-07-25 00:00:...|       2911|     PROCESSING|
|       9|2013-07-25 00:00:...|       5657|PENDING_PAYMENT|
|      10|2013-07-25 00:00:...|       5648|PENDING_PAYMENT|
|      11|2013-07-25 00:00:...|        918| PAYMENT_REVIEW|
|      12|2013-07-25 00:00:...|       1837|         CLOSED|
|      13|2013-07-25 00:00:...|       9149|PENDING_PAYMENT|
|      14|2013-07-25 00:00:...|       98

In [32]:
# spark.sql("""
#   CREATE TABLE my_db_spark.orders2 (
#     order_id integer,
#     order_date string,
#     customer_id integer,
#     order_status string
#   )
#   USING csv
#   OPTIONS (
#     'path' 's3://fcc-spark-example/dataset/2023/orders.csv',
#     'header' 'true',
#     'sep' ',',
#     'inferSchema' 'true',
#     'mode' 'FAILFAST',
#     'quote' '"',
#     'escape' '"',
#     'multiline' 'true',
#     'charset' 'UTF-8'
#   )
# """)


In [70]:
spark.sql('show tables').show()

+-----------+---------+-----------+
|  namespace|tableName|isTemporary|
+-----------+---------+-----------+
|my_db_spark|   orders|      false|
+-----------+---------+-----------+



In [71]:
spark.sql('DESCRIBE EXTENDED orders').show()

+--------------------+--------------------+-------+
|            col_name|           data_type|comment|
+--------------------+--------------------+-------+
|            order_id|                 int|   null|
|          order_date|              string|   null|
|         customer_id|                 int|   null|
|        order_status|              string|   null|
|                    |                    |       |
|# Detailed Table ...|                    |       |
|            Database|         my_db_spark|       |
|               Table|              orders|       |
|               Owner|              hadoop|       |
|        Created Time|Sun May 14 02:30:...|       |
|         Last Access|             UNKNOWN|       |
|          Created By|  Spark 3.3.0-amzn-1|       |
|                Type|            EXTERNAL|       |
|            Provider|             parquet|       |
|            Location|s3://fcc-spark-ex...|       |
|       Serde Library|org.apache.hadoop...|       |
|         In

In [35]:
## This would FAIL 

# spark.sql('TRUNCATE table orders')

### DML Operations 
    - INSERT - it works -> But its mostly for OLTP application, its not for Spark ideally
    - UPDATE - doesnt work 
    - DELETE - doesnt work 
    - SELECT - Always work 
    
    (These are as part of Open Source Apache Spark)

In [72]:
spark.sql("INSERT INTO TABLE my_db_spark.orders VALUES (9988, '2023-05-23', 1234, 'COMPLETED')")

DataFrame[]

In [73]:
spark.sql("INSERT INTO TABLE my_db_spark.orders VALUES (9989, '2023-05-23', 1235, 'COMPLETED')")

DataFrame[]

In [74]:
# We will not see any data here 

!hadoop fs -ls hdfs://ip-172-31-2-35.us-east-2.compute.internal:8020/user/spark/warehouse/my_db_spark.db/

In [75]:
spark.sql('SELECT * FROM my_db_spark.orders WHERE order_id IN (9988, 9989)').show()

+--------+--------------------+-----------+---------------+
|order_id|          order_date|customer_id|   order_status|
+--------+--------------------+-----------+---------------+
|    9988|2013-09-25 00:00:...|      11739|SUSPECTED_FRAUD|
|    9989|2013-09-25 00:00:...|       4865|       COMPLETE|
|    9989|          2023-05-23|       1235|      COMPLETED|
|    9988|          2023-05-23|       1234|      COMPLETED|
+--------+--------------------+-----------+---------------+

